In [1]:
import torch.nn as nn
from tqdm import tqdm
import torch
import numpy as np
from sklearn.metrics import roc_auc_score, roc_curve, auc
import wandb
import pandas as pd
from sklearn.model_selection import KFold

import pickle
import timeit
import os
import random
import glob
import pandas as pd
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

from sklearn.metrics import roc_curve, auc


class ProteinProteinInteractionPrediction(nn.Module):
    def __init__(self,mod_embed,prot_embed,dim=20,layer_gnn=2):
        super(ProteinProteinInteractionPrediction, self).__init__()
        self.prot_embed_fingerprint = nn.Embedding(n_fingerprint, dim)
        self.mod_embed_fingerprint = nn.Embedding(nmod_fingerprint, dim)
        self.mod_W_gnn = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layer_gnn)])
        self.prot_gnn = nn.ModuleList([nn.Linear(dim, dim) for _ in range(layer_gnn)])
        self.prot_W1_attention = nn.Linear(dim, dim)
        self.prot_W2_attention = nn.Linear(dim, dim)
        self.prot_w = nn.Parameter(torch.zeros(dim))
        self.W1_attention = nn.Linear(dim, dim)
        self.W2_attention = nn.Linear(2*dim, dim)  # Modified to accept concatenated protein vector
        self.w = nn.Parameter(torch.zeros(dim))
        self.W_out = nn.Sequential(
            nn.Linear(2*dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64,16),
            nn.ReLU(),
            nn.Linear(16, 2)
        )
        
        self.mod_embed = mod_embed
        self.prot_embed = prot_embed

        self.rdkit_linear = nn.Linear(2048, dim)

    def gnn(self, xs1, A1, type):
        for i in range(layer_gnn):
            if type == "mod":
                hs1 = torch.relu(self.mod_W_gnn[i](xs1))
            elif type == "prot":
                hs1 = torch.relu(self.prot_gnn[i](xs1))
            xs1 = torch.matmul(A1, hs1)

        return xs1
    
    def mutual_attention(self, h1, h2):
        x1 = self.W1_attention(h1)
        x2 = self.W2_attention(h2)

        m1 = x1.size()[0]
        m2 = x2.size()[0]

        c1 = x1.repeat(1, m2).view(m1, m2, dim)
        c2 = x2.repeat(m1, 1).view(m1, m2, dim)

        d = torch.tanh(c1 + c2)
        alpha = torch.matmul(d, self.w).view(m1, m2)

        b1 = torch.mean(alpha, 1)
        p1 = torch.softmax(b1, 0)
        s1 = torch.matmul(torch.t(x1), p1).view(-1, 1)

        b2 = torch.mean(alpha, 0)
        p2 = torch.softmax(b2, 0)
        s2 = torch.matmul(torch.t(x2), p2).view(-1, 1)

        return torch.cat((s1, s2), 0).view(1, -1), p1, p2
    def prot_mutual_attention(self, h1, h2):
        x1 = self.prot_W1_attention(h1)
        x2 = self.prot_W2_attention(h2)

        m1 = x1.size()[0]
        m2 = x2.size()[0]

        c1 = x1.repeat(1, m2).view(m1, m2, dim)
        c2 = x2.repeat(m1, 1).view(m1, m2, dim)

        d = torch.tanh(c1 + c2)
        alpha = torch.matmul(d, self.prot_w).view(m1, m2)

        b1 = torch.mean(alpha, 1)
        p1 = torch.softmax(b1, 0)
        s1 = torch.matmul(torch.t(x1), p1).view(-1, 1)

        b2 = torch.mean(alpha, 0)
        p2 = torch.softmax(b2, 0)
        s2 = torch.matmul(torch.t(x2), p2).view(-1, 1)

        return torch.cat((s1, s2), 0).view(1, -1), p1, p2

    def forward(self, inputs, train = True):
        fingerprints1, adjacency1, fingerprints2, adjacency2, fingerprints3, adjacency3, smiles = inputs

        """Protein vector with GNN."""
        x_fingerprints1 = self.mod_embed_fingerprint(fingerprints1)
        x_fingerprints2 = self.prot_embed_fingerprint(fingerprints2)
        x_fingerprints3 = self.prot_embed_fingerprint(fingerprints3)
        if self.mod_embed == "gnn":
            x_mod = self.gnn(x_fingerprints1, adjacency1, "mod")
        elif self.mod_embed == "rdkit":
            # Implement RDKit molecular fingerprinting
            from rdkit import Chem
            from rdkit.Chem import rdFingerprintGenerator

            # Convert SMILES to RDKit mol object
            mol = Chem.MolFromSmiles(smiles)
            # Generate Morgan fingerprint
            morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius=2,fpSize=2048)
            morgan_fp = morgan_gen.GetFingerprint(mol)
            # Convert bit vector to PyTorch tensor
            x_mod = torch.tensor([int(b) for b in morgan_fp.ToBitString()]).float().view(1, -1)
            x_mod = self.rdkit_linear(x_mod)            
        elif self.mod_embed == "graphmvp":
            x_mod = self.graphmvp_embed(fingerprints1)
        elif self.mod_embed == "infograph":
            x_mod = self.infograph_embed(fingerprints1)

        if self.prot_embed == "gnn":
            x_protein2 = self.gnn(x_fingerprints2, adjacency2, "prot")
            x_protein3 = self.gnn(x_fingerprints3, adjacency3, "prot")
        # x_mod = N, dim

        """Concatenate protein vectors"""
        x_proteins, p1, p2 = self.prot_mutual_attention(x_protein2, x_protein3)
        """Protein vector with mutual-attention."""
        y, p1, p2 = self.mutual_attention(x_mod, x_proteins)

        z_interaction = self.W_out(y)
        # ADD CHEMICAL FEATURE VECTOR HERE 
        return z_interaction, p1, p2

    def __call__(self, data, train=True):
        inputs, t_interaction_list = data[:-1], data[-1]
        # inputs = list(zip(*inputs))
        # print(inputs[0])

        z_interaction_list = []
        import concurrent.futures
        from concurrent.futures import ThreadPoolExecutor   
        
        with ThreadPoolExecutor(max_workers=64) as executor:
            # future = executor.submit(self.forward, inputs, train)
            futures = {executor.submit(self.forward, input, train): idx for idx, input in enumerate(zip(*inputs))}
            for future in concurrent.futures.as_completed(futures):
                idx = futures[future]
                z_interaction, p1, p2 = future.result()
                z_interaction_list.insert(idx, z_interaction)
        
        if train:
            z_interaction_list = torch.stack(z_interaction_list,dim=1).squeeze(0).float()
            t_interaction_list = torch.nn.functional.one_hot(t_interaction_list, num_classes=2).float()
            # print(z_interaction_list.shape, t_interaction_list.shape)
            loss = F.cross_entropy(z_interaction_list, t_interaction_list)
            return loss
        else:
            z_interaction_list = torch.stack(z_interaction_list, dim=1).squeeze(0).float()
            t_interaction_list = t_interaction_list.to("cpu").data.numpy()
            z = F.softmax(z_interaction_list, 1).to("cpu").data.numpy()
            return z, t_interaction_list, p1, p2

In [2]:
class Trainer(object):
    def __init__(self, model, lr=1e-4, train_size=3000, batch_size=32):
        self.model = model
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr)
        self.train_size = train_size
        self.batch_size = batch_size

    def train(self, dataset):
        self.model.train()  # Set model to training mode
        sampling = random.choices(dataset, k=self.train_size)
        loss_total = 0

        for i in tqdm(range(0, len(sampling), self.batch_size)):
            batch = sampling[i:i + self.batch_size]
            mod_fps, mod_adjs, proteins1, adjacencies1, proteins2, adjacencies2, smiles_list, interactions = [], [], [], [], [], [], [], []

            for data in batch:
                try:
                    mod, p1, p2, interaction, family = data
                    A1 = np.load(
                        prot_data[prot_data['protein'] == p1].iloc[0]['adj_path'],
                        allow_pickle=True
                    )
                    A2 = np.load(
                        prot_data[prot_data['protein'] == p2].iloc[0]['adj_path'],
                        allow_pickle=True
                    )
                    P1 = np.load(
                        prot_data[prot_data['protein'] == p1].iloc[0]['fp_path'],
                        allow_pickle=True
                    )
                    P2 = np.load(
                        prot_data[prot_data['protein'] == p2].iloc[0]['fp_path'],
                        allow_pickle=True
                    )
                    mod_fp = np.load(
                        mod_data[mod_data['inchikey'] == mod].iloc[0]['fp_path'],
                        allow_pickle=True
                    )
                    mod_adj = np.load(
                        mod_data[mod_data['inchikey'] == mod].iloc[0]['adj_path'],
                        allow_pickle=True
                    )
                    smiles = mod_data[mod_data['inchikey'] == mod].iloc[0]['smiles']
                except Exception as e:
                    # print(f"failed for {e}")
                    continue

                proteins1.append(torch.LongTensor(P1.astype(np.float32)))
                proteins2.append(torch.LongTensor(P2.astype(np.float32)))
                adjacencies1.append(torch.FloatTensor(A1.astype(np.float32)))
                adjacencies2.append(torch.FloatTensor(A2.astype(np.float32)))
                mod_fps.append(torch.LongTensor(mod_fp.astype(np.float32)))
                mod_adjs.append(torch.FloatTensor(mod_adj.astype(np.float32)))
                smiles_list.append(smiles)
                interactions.append(torch.LongTensor([interaction.astype(int)]))

            if len(proteins1) == 0:
                continue

            proteins1 = [p.to(device) for p in proteins1]
            proteins2 = [p.to(device) for p in proteins2]
            adjacencies1 = [a.to(device) for a in adjacencies1]
            adjacencies2 = [a.to(device) for a in adjacencies2]
            mod_fps = [m.to(device) for m in mod_fps]
            mod_adjs = [m.to(device) for m in mod_adjs]
            interactions = torch.cat(interactions).to(device)

            comb = (mod_fps, mod_adjs, proteins1, adjacencies1, proteins2, adjacencies2, smiles_list, interactions)
            loss = self.model(comb)
            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            loss_total += loss.item()

        return loss_total / len(sampling)


In [3]:


class Tester(object):
    def __init__(self, model, batch_size = 10, test_size = 1000, wandb_project_name=False):
        self.model = model
        self.test_size = test_size
        self.batch_size = batch_size
    def test(self, dataset, epoch=None):
        sampling = random.choices(dataset, k=self.test_size)
        z_list, t_list = [], []
        
        for i in tqdm(range(0, len(sampling), self.batch_size)):
            batch = sampling[i:i + self.batch_size]
            mod_fps, mod_adjs, proteins1, adjacencies1, proteins2, adjacencies2, smiles_list, interactions = [], [], [], [], [], [], [], []

            for data in batch:
                try:
                    mod, p1, p2, interaction, _ = data
                    A1 = np.load(
                        prot_data[prot_data['protein'] == p1].iloc[0]['adj_path'],
                        allow_pickle=True
                    )
                    A2 = np.load(
                        prot_data[prot_data['protein'] == p2].iloc[0]['adj_path'],
                        allow_pickle=True
                    )
                    P1 = np.load(
                        prot_data[prot_data['protein'] == p1].iloc[0]['fp_path'],
                        allow_pickle=True
                    )
                    P2 = np.load(
                        prot_data[prot_data['protein'] == p2].iloc[0]['fp_path'],
                        allow_pickle=True
                    )
                    mod_fp = np.load(
                        mod_data[mod_data['inchikey'] == mod].iloc[0]['fp_path'],
                        allow_pickle=True
                    )
                    mod_adj = np.load(
                        mod_data[mod_data['inchikey'] == mod].iloc[0]['adj_path'],
                        allow_pickle=True
                    )
                    smiles = mod_data[mod_data['inchikey'] == mod].iloc[0]['smiles']
                except Exception as e:
                    # print(f"failed for {mod}, {p1}, and {p2}: {e}")
                    continue

                proteins1.append(torch.LongTensor(P1.astype(np.float32)))
                proteins2.append(torch.LongTensor(P2.astype(np.float32)))
                adjacencies1.append(torch.FloatTensor(A1.astype(np.float32)))
                adjacencies2.append(torch.FloatTensor(A2.astype(np.float32)))
                mod_fps.append(torch.LongTensor(mod_fp.astype(np.float32)))
                mod_adjs.append(torch.FloatTensor(mod_adj.astype(np.float32)))
                smiles_list.append(smiles)
                interactions.append(torch.LongTensor([interaction.astype(int)]))

            if len(proteins1) == 0:
                continue

            proteins1 = [p.to(device) for p in proteins1]
            proteins2 = [p.to(device) for p in proteins2]
            adjacencies1 = [a.to(device) for a in adjacencies1]
            adjacencies2 = [a.to(device) for a in adjacencies2]
            mod_fps = [m.to(device) for m in mod_fps]
            mod_adjs = [m.to(device) for m in mod_adjs]
            interactions = torch.cat(interactions).to(device)

            comb = (mod_fps, mod_adjs, proteins1, adjacencies1, proteins2, adjacencies2, smiles_list, interactions)
            with torch.no_grad():
                z, _, _, _ = self.model(comb, train=False)
            z_list.extend(z)
            t_list.extend(interactions)

        score_list, label_list = [], []
        for z in z_list:
            score_list.append(z[1].item())
            label_list.append(torch.argmax(torch.FloatTensor(z)).item())

        labels = np.array(label_list)
        y_true = np.array([t.item() for t in t_list])
        y_pred = np.array(score_list)

        (
            tp,
            fp,
            tn,
            fn,
            accuracy,
            precision,
            sensitivity,
            recall,
            specificity,
            MCC,
            F1_score,
            Q9,
            ppv,
            npv,
        ) = self.calculate_performace(len(sampling), labels, y_true)
        roc_auc_val = roc_auc_score(y_true, y_pred)
        fpr, tpr, thresholds = roc_curve(y_true, y_pred)
        auc_val = auc(fpr, tpr)

        # Log results to wandb
        # if self.use_wandb:
        #     wandb.log({
        #         "Epoch": epoch,
        #         "Accuracy": accuracy,
        #         "Precision": precision,
        #         "Recall": recall,
        #         "Sensitivity": sensitivity,
        #         "Specificity": specificity,
        #         "MCC": MCC,
        #         "F1 Score": F1_score,
        #         "ROC AUC": roc_auc_val,
        #         "AUC": auc_val,
        #         "TP": tp,
        #         "FP": fp,
        #         "TN": tn,
        #         "FN": fn,
        #         "PPV": ppv,
        #         "NPV": npv,
        #     })

        return (
            accuracy,
            precision,
            recall,
            sensitivity,
            specificity,
            MCC,
            F1_score,
            roc_auc_val,
            auc_val,
            Q9,
            ppv,
            npv,
            tp,
            fp,
            tn,
            fn,
        )

    def result(
        self,
        epoch,
        time,
        loss,
        accuracy,
        precision,
        recall,
        sensitivity,
        specificity,
        MCC,
        F1_score,
        roc_auc_val,
        auc_val,
        Q9,
        ppv,
        npv,
        tp,
        fp,
        tn,
        fn,
        file_name,
    ):
        import os

        # Ensure the directory exists
        os.makedirs(os.path.dirname(file_name), exist_ok=True)

        # Open the file in append mode, creating it if it doesn't exist
        with open(file_name, "a+") as f:
            result = map(
                str,
                [
                    epoch,
                    time,
                    loss,
                    accuracy,
                    precision,
                    recall,
                    sensitivity,
                    specificity,
                    MCC,
                    F1_score,
                    roc_auc_val,
                    auc_val,
                    Q9,
                    ppv,
                    npv,
                    tp,
                    fp,
                    tn,
                    fn,
                ],
            )
            f.write("\t".join(result) + "\n")

    def calculate_performace(self, test_num, pred_y, labels):
        tp = 0
        fp = 0
        tn = 0
        fn = 0
        test_num = len(pred_y)
        for index in range(test_num):
            if labels[index] == 1:
                if labels[index] == pred_y[index]:
                    tp = tp + 1
                else:
                    fn = fn + 1
            else:
                if labels[index] == pred_y[index]:
                    tn = tn + 1
                else:
                    fp = fp + 1

        if (tp + fn) == 0:
            q9 = float(tn - fp) / (tn + fp + 1e-06)
        if (tn + fp) == 0:
            q9 = float(tp - fn) / (tp + fn + 1e-06)
        if (tp + fn) != 0 and (tn + fp) != 0:
            q9 = 1 - float(np.sqrt(2)) * np.sqrt(float(fn * fn) / ((tp + fn) * (tp + fn)) + float(fp * fp) / ((tn + fp) * (tn + fp)))

        Q9 = (float)(1 + q9) / 2
        accuracy = float(tp + tn) / test_num
        precision = float(tp) / (tp + fp + 1e-06)
        sensitivity = float(tp) / (tp + fn + 1e-06)
        recall = float(tp) / (tp + fn + 1e-06)
        specificity = float(tn) / (tn + fp + 1e-06)
        ppv = float(tp) / (tp + fp + 1e-06)
        npv = float(tn) / (tn + fn + 1e-06)
        F1_score = float(2 * tp) / (2 * tp + fp + fn + 1e-06)
        MCC = float(tp * tn - fp * fn) / (np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)))

        return tp, fp, tn, fn, accuracy, precision, sensitivity, recall, specificity, MCC, F1_score, Q9, ppv, npv

    def save_model(self, model, file_name):
        torch.save(model.state_dict(), file_name)

In [4]:
z = torch.FloatTensor(np.array([0.1, 0.2], dtype=np.float32))
print(z, torch.argmax(z,dim=0))

tensor([0.1000, 0.2000]) tensor(1)


In [5]:

# get data
prot_data = pd.read_csv("prot_data.csv")
mod_data = pd.read_csv("mod_data.csv")
train_data = pd.read_csv("interaction_data.csv")
examples = np.array(train_data.values.tolist())
# examples = np.array(random.choices(examples_all, k=500))

# setup folders
prot_fp_folder = "protein_fingerprints"
mod_fp_folder = "mod_fingerprints"
prot_fp_dict = np.load("protein_fingerprints/prot_fingerprint_dict.pickle",allow_pickle=True)
mod_fp_dict = np.load("mod_fingerprints/mod_fingerprint_dict.pickle",allow_pickle=True)


n_fingerprint = len(prot_fp_dict) + 100
nmod_fingerprint = len(mod_fp_dict) + 100


### Hyperparameters ###

radius         = 1
dim        = 20
layer_gnn      = 2
lr             = 1e-4
lr_decay       = 0.5
decay_interval = 10
iteration      = 30

import wandb
import timeit
import torch

fold_count = 1
### Check if GPU is available ###
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

### Define k-folds ###
num_kfolds = 5
kfold      = KFold(n_splits=num_kfolds, shuffle=True, random_state=1)  # Updated


In [6]:
train_size = 3
test_size = 1000
batch_size = 10
log_wandb = True

for train, test in kfold.split(examples):
    dataset_train = examples[train]  # mod, prot1, prot2, int, int_family
    dataset_test = examples[test]

    start = timeit.default_timer()

    model = ProteinProteinInteractionPrediction(
        dim=dim, 
        layer_gnn=layer_gnn, 
        mod_embed="rdkit", 
        prot_embed = "gnn",
    ).to(device)
    trainer = Trainer(model, lr = lr, train_size=train_size, batch_size=batch_size)
    file_model = "ppim/model/" + "sep2_model_fold_" + str(fold_count)
    file_result = "ppim/result/" + "sep2_results_fold_" + str(fold_count) + ".txt"
    if log_wandb:
        wandb.init(project="promisegat4")
        # Log file paths to wandb
        wandb.config.update({
            "fold": fold_count,
            "model_path": file_model,
            "result_path": file_result,
            "protein_embed": "gcn",
            "mod_embed": "gcn",
            "radius": radius,
            "dim": dim,
            "layer_gnn": layer_gnn,
            "lr": lr,
            "lr_decay": lr_decay,
            "decay_interval": decay_interval,
            "iteration": iteration
        })

    for epoch in range(iteration):
        if (epoch + 1) % decay_interval == 0:
            trainer.optimizer.param_groups[0]["lr"] *= lr_decay

        loss = trainer.train(dataset_train)
        print(f"finished with loss {loss}")

        # Log training loss and GPU usage to wandb
        gpu_usage = torch.cuda.memory_allocated(device=device) / 1024 ** 3  # in GB
        if log_wandb:
            wandb.log({"epoch": epoch, "loss": loss, "gpu_usage_gb": gpu_usage, "fold": fold_count})

        tester = Tester(model, test_size = test_size,batch_size=batch_size)
        (
            accuracy,
            precision,
            recall,
            sensitivity,
            specificity,
            MCC,
            F1_score,
            roc_auc_val,
            auc_val,
            Q9,
            ppv,
            npv,
            tp,
            fp,
            tn,
            fn,
        ) = tester.test(dataset_test, epoch=epoch)
        end = timeit.default_timer()
        time = end - start

        # Log results to wandb
        if log_wandb:
            wandb.log({
            "epoch": epoch,
            "time": time,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "sensitivity": sensitivity,
            "specificity": specificity,
            "MCC": MCC,
            "F1_score": F1_score,
            "ROC_AUC": roc_auc_val,
            "AUC": auc_val,
            "Q9": Q9,
            "PPV": ppv,
            "NPV": npv,
            "TP": tp,
            "FP": fp,
            "TN": tn,
            "FN": fn,
            "fold": fold_count
        })

        tester.result(
            epoch,
            time,
            loss,
            accuracy,
            precision,
            recall,
            sensitivity,
            specificity,
            MCC,
            F1_score,
            roc_auc_val,
            auc_val,
            Q9,
            ppv,
            npv,
            tp,
            fp,
            tn,
            fn,
            file_result,
        )
        tester.save_model(model, file_model)


        print("Epoch: " + str(epoch))
        print("Accuracy: " + str(accuracy))
        print("Precision: " + str(precision))
        print("Recall: " + str(recall))
        print("Sensitivity: " + str(sensitivity))
        print("Specificity: " + str(specificity))
        print("MCC: " + str(MCC))
        print("F1-score: " + str(F1_score))
        print("ROC-AUC: " + str(roc_auc_val))
        print("AUC: " + str(auc_val))
        print("Q9: " + str(Q9))
        print("PPV: " + str(ppv))
        print("NPV: " + str(npv))
        print("TP: " + str(tp))
        print("FP: " + str(fp))
        print("TN: " + str(tn))
        print("FN: " + str(fn))
        print("\n")

        torch.manual_seed(1234)
    if log_wandb:
        wandb.finish()
    fold_count += 1

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: spycoderyt. Use `wandb login --relogin` to force relogin


100%|██████████| 1/1 [00:00<00:00,  1.07it/s]


finished with loss 0.2495382030804952


100%|██████████| 100/100 [00:37<00:00,  2.70it/s]
/var/folders/d3/q38hjcyx7tj50fgmqb4mcm8r0000gn/T/ipykernel_55766/1284111422.py:231: RuntimeWarning: invalid value encountered in scalar divide
  MCC = float(tp * tn - fp * fn) / (np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)))


Epoch: 0
Accuracy: 0.37681159420289856
Precision: 0.37681159378281875
Recall: 0.9999999970414202
Sensitivity: 0.9999999970414202
Specificity: 0.0
MCC: nan
F1-score: 0.5473684206094183
ROC-AUC: 0.5069598077716971
AUC: 0.5069598077716971
Q9: 0.2928932188134524
PPV: 0.37681159378281875
NPV: 0.0
TP: 338
FP: 559
TN: 0
FN: 0




...

100%|██████████| 100/100 [00:45<00:00,  2.19it/s]
/var/folders/d3/q38hjcyx7tj50fgmqb4mcm8r0000gn/T/ipykernel_55766/1284111422.py:231: RuntimeWarning: invalid value encountered in scalar divide
  MCC = float(tp * tn - fp * fn) / (np.sqrt((tp + fp) * (tp + fn) * (tn + fp) * (tn + fn)))


Epoch: 24
Accuracy: 0.6331828442437923
Precision: 0.0
Recall: 0.0
Sensitivity: 0.0
Specificity: 0.9999999982174688
MCC: nan
F1-score: 0.0
ROC-AUC: 0.5325709584533114
AUC: 0.5325709584533114
Q9: 0.2928932188134524
PPV: 0.0
NPV: 0.633182843529139
TP: 0
FP: 0
TN: 561
FN: 325




100%|██████████| 1/1 [00:00<00:00,  2.51it/s]


finished with loss 0.24011355638504028


 73%|███████▎  | 73/100 [00:29<00:11,  2.41it/s]